In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

# получим переменные из .env
load_dotenv()

client = OpenAI()

# Создание Вакансий

In [40]:
system_prompt = '''
Ты директор IT компании которая разрабатывает сайты с применением языковых моделей LLM. 
Проанализируй структуру it-отдела разработки сайтов из 15 человек и добавь наименование 10 недостающих должностей: 

Разработчик Python, 
Аналитик данных, 
Инженер по машинному обучению, 
Web-разработчик, 
Системный администратор

Для информации: Ruby мы не используем
Наш стек: Python, Django, Angular, MogoDB, Docker
'''

completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Ответ дай без пояснений в виде перечня из 15 должностей:  Пример ответа: Разработчик Python,..."}
  ]
)

In [41]:
print(completion.choices[0].message.content)
vacancies = list(completion.choices[0].message.content.split(","))

Разработчик Python, 
Аналитик данных, 
Инженер по машинному обучению, 
Web-разработчик, 
Системный администратор,
Базы данных MongoDB - Инженер,
DevOps инженер,
Full Stack Developer,
Front-End разработчик (Angular),
Back-End разработчик (Django),
Data Scientist,
UI/UX дизайнер,
Технический писатель,
Инженер по тестированию,
DevSecOps инженер.


In [ ]:
file_name = 'vacancies.txt'

with open(file_name, 'w', encoding='utf-8') as file:
    for vacancy in vacancies:
        file.write(vacancy)

In [57]:
import json

def print_pretty_json(json_obj):
    """
    Красиво печатает JSON объект.
    :param json_obj: JSON объект для печати (в Python обычно это словарь).
    """
    pretty_json = json.dumps(json_obj, indent=4, ensure_ascii=False)
    print(pretty_json)

In [75]:
system_prompt_vac = '''
Ты HR специалист с IT бэкграундом в компании
которая разрабатывает сайты с применением языковых моделей LLM. 
Наш стек для програмистов: Python, Django, Angular, MogoDB, Docker
По представленному названию должности составь JSON объект:

"title": "Название вакансии." 
"description": "Краткое описание вакансии.",
"requirements": ["Требование к вакансии 1", "Требование к вакансии 2",...]

Все описания должны быть на русском языке.
'''

import json
from tqdm import tqdm

MAX_LENGTH = 2048 
vacancies_data=[]

for vac in tqdm(vacancies, desc="Обработка вакансий"):
    for attempt in range(3):  # Повторять до 3 раз
        try:
            completion = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": system_prompt_vac},
                    {"role": "user", "content": f"Вакансия: {vac}"}
                ]
            )
            temp_data = json.loads(completion.choices[0].message.content)
            
            # Проверка длины текста
            combined_text = temp_data["title"] + temp_data["description"] + ''.join(temp_data["requirements"])
            if len(combined_text) > MAX_LENGTH:
                raise ValueError(f"Длина текста превышает {MAX_LENGTH} символов")
            
            vacancies_data.append(temp_data)
            break  # Прервать цикл при успешном выполнении
        except Exception as e:
            print(f"Произошла ошибка: {e}. Попытка {attempt + 1} из 3.")
            if attempt == 2:  # Если это последняя попытка
                print("Не удалось обработать вакансию после 3 попыток.")



Обработка вакансий: 100%|██████████| 15/15 [07:17<00:00, 29.14s/it]


In [76]:
print_pretty_json(vacancies_data[0])
print_pretty_json(vacancies_data[1])

{
    "title": "Разработчик Python",
    "description": "Мы ищем опытного разработчика Python для работы над усовершенствованием и разработкой наших сайтов с применением языковых моделей LLM.",
    "requirements": [
        "Бакалавр, специалист или магистр в области информационных технологий, науки о данных или смежной области",
        "Минимум 3 года опыта работы с Python",
        "Опыт разработки на Django",
        "Иметь знания/опыт работы с MongoDB",
        "Владение Docker",
        "Понимание и применение Agile методологии",
        "Способность работать в команде, решать проблемы и осуществлять многозадачность",
        "Хорошие навыки общения на русском языке"
    ]
}
{
    "title": "Аналитик данных",
    "description": "Аналитик данных будет работать на пересечении IT и аналитики бизнеса, используя свои навыки обработки данных с целью преобразования больших объемов информации в управляемые и полезные входные данные для нашей команды по разработке модели LLM.",
    "requir

In [77]:
with open('vacancies.json', 'w', encoding='utf-8') as file:
    json.dump(vacancies_data, file, ensure_ascii=False, indent=4)

In [109]:
import sqlite3

conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS vacancies (
        id INTEGER PRIMARY KEY,
        title TEXT,
        description TEXT,
        requirements TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS resumes (
        id INTEGER PRIMARY KEY,
        skills TEXT,
        experience TEXT
    )
''')

conn.commit()
conn.close()

In [80]:
conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

for vacancy in vacancies_data:
    cursor.execute('''
        INSERT INTO vacancies (title, description, requirements)
        VALUES (?, ?, ?)
    ''', (vacancy["title"], vacancy["description"], ", ".join(vacancy["requirements"])))

conn.commit()
conn.close()

In [81]:
import sqlite3

conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM vacancies LIMIT 5')
rows = cursor.fetchall()

conn.close()

# Вывод результатов
for row in rows:
    print(f"ID: {row[0]}, Title: {row[1]}, Description: {row[2]}, Requirements: {row[3]}")


ID: 1, Title: Разработчик Python, Description: Мы ищем опытного разработчика Python для работы над усовершенствованием и разработкой наших сайтов с применением языковых моделей LLM., Requirements: Бакалавр, специалист или магистр в области информационных технологий, науки о данных или смежной области, Минимум 3 года опыта работы с Python, Опыт разработки на Django, Иметь знания/опыт работы с MongoDB, Владение Docker, Понимание и применение Agile методологии, Способность работать в команде, решать проблемы и осуществлять многозадачность, Хорошие навыки общения на русском языке
ID: 2, Title: Аналитик данных, Description: Аналитик данных будет работать на пересечении IT и аналитики бизнеса, используя свои навыки обработки данных с целью преобразования больших объемов информации в управляемые и полезные входные данные для нашей команды по разработке модели LLM., Requirements: Высшее техническое образование, Опыт работы с Python и SQL (MongoDB), Навыки работы с большими данными и опыт их ан

In [114]:
import pandas as pd

conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM vacancies')
rows = cursor.fetchall()

conn.close()

df_vacancies = pd.DataFrame(rows, columns=['id', 'title', 'description','requirements'])
df_vacancies.to_excel('vacancies.xlsx', index=False)

# Создание Резюме

In [93]:
system_prompt_vac = '''
Ты специалист по подготовке данных с IT бэкграундом в компании
которая разрабатывает сайты с применением языковых моделей LLM. 
Наш стек для програмистов: Python, Django, Angular, MogoDB, Docker
По представленному названию вакансии сгенерируй резюме предполагаемого кандидата в JSON объект,
названия компаний придумывай опираясь на знание Российского it-рынка,
обязательно указывай уровень английского языка:

"skills": ["Навык 1", "Навык 2", ...], 
"experience": ["Опыт работы в компании 1", "Опыт работы в компании 2",...]

Все описания должны быть на русском языке.
'''

import json
from tqdm import tqdm

MAX_LENGTH = 2048 
resume_data=[]

for vac in tqdm(vacancies, desc="Обработка резюме"):
    for attempt in range(3):  # Повторять до 3 раз
        try:
            completion = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": system_prompt_vac},
                    {"role": "user", "content": f"Вакансия: {vac}"}
                ]
            )
            temp_data = json.loads(completion.choices[0].message.content)
            
            # Проверка длины текста
            combined_text =  ''.join(temp_data["skills"]) + ''.join(temp_data["experience"])
            if len(combined_text) > MAX_LENGTH:
                raise ValueError(f"Длина текста превышает {MAX_LENGTH} символов")
            
            resume_data.append(temp_data)
            break  # Прервать цикл при успешном выполнении
        except Exception as e:
            print(f"Произошла ошибка: {e}. Попытка {attempt + 1} из 3.")
            if attempt == 2:  # Если это последняя попытка
                print("Не удалось обработать резюме после 3 попыток.")

Обработка резюме:  67%|██████▋   | 10/15 [06:19<03:11, 38.20s/it]

Произошла ошибка: Expecting value: line 1 column 1 (char 0). Попытка 1 из 3.


Обработка резюме:  73%|███████▎  | 11/15 [07:24<03:05, 46.44s/it]

Произошла ошибка: 'skills'. Попытка 1 из 3.
Произошла ошибка: Expecting value: line 1 column 1 (char 0). Попытка 2 из 3.


Обработка резюме: 100%|██████████| 15/15 [10:24<00:00, 41.63s/it]


In [95]:
with open('resumes.json', 'w', encoding='utf-8') as file:
    json.dump(resume_data, file, ensure_ascii=False, indent=4)

In [96]:
print_pretty_json(resume_data[0])

{
    "profile": {
        "name": "Иван Иванов",
        "email": "ivanov@gmail.com",
        "phone": "+7(999)999-9999"
    },
    "professional_title": "Python разработчик",
    "skills": [
        "Знание Python и стандартной библиотеки Python",
        "Опыт работы с Django",
        "Владение SQL и опыт работы с СУБД",
        "Понимание принципов разработки программного обеспечения",
        "Использование систем контроля версий, например, Git",
        "Английский язык повышенного уровня для чтения технической документации",
        "Опыт работы в Docker"
    ],
    "experience": [
        "OOО 'Техлогия', Python разработчик. Разработка и поддержка внутренних веб-сервисов компании на Django. Занимался оптимизацией работы с БД, рефакторингом кода.",
        "ЗАО 'Дигиспейс', Junior Python разработчик. Вел разработку модулей на Python для управления системами автоматизации."
    ],
    "languages": [
        "Русский - родной",
        "Английский - B2"
    ]
}


In [110]:
conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

for resume in resume_data:
    cursor.execute('''
        INSERT INTO resumes (skills, experience)
        VALUES (?, ?)
    ''', ("; ".join(resume["skills"]), "; ".join(resume["experience"])))

conn.commit()
conn.close()

In [111]:
conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM resumes LIMIT 5')
rows = cursor.fetchall()

conn.close()

# Вывод результатов
for row in rows:
    print(f"ID: {row[0]}, skills: {row[1]}, experience: {row[2]}")

ID: 1, skills: Знание Python и стандартной библиотеки Python; Опыт работы с Django; Владение SQL и опыт работы с СУБД; Понимание принципов разработки программного обеспечения; Использование систем контроля версий, например, Git; Английский язык повышенного уровня для чтения технической документации; Опыт работы в Docker, experience: OOО 'Техлогия', Python разработчик. Разработка и поддержка внутренних веб-сервисов компании на Django. Занимался оптимизацией работы с БД, рефакторингом кода.; ЗАО 'Дигиспейс', Junior Python разработчик. Вел разработку модулей на Python для управления системами автоматизации.
ID: 2, skills: Умение работать с большими данными; Знание Python, pandas, NumPy, MongoDB; Знание SQL, PostgreSQL; Умение строить модели машинного обучения; Визуализация данных с использованием matplotlib, seaborn; Уверенное владение Excel; Умение работать с BI системами, Tableau, experience: «1С-Битрикс»: анализ структуры данных, генерация отчетов, работа с большими данными; «Яндекс»: 

In [113]:
conn = sqlite3.connect('vacancies.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM resumes')
rows = cursor.fetchall()

conn.close()

df_resumes = pd.DataFrame(rows, columns=['id', 'skills', 'experience'])
df_resumes.to_excel('resumes.xlsx', index=False)